# NR preprocessing

* This notebooks gets the Boa output file and convet it to a dictionary like for checking inconsistencies in the annotations


## Convert Boa output to dictionary like index

* Boa output as an input to python script

```
count[12E8][12E8][10090] = 4
count[12E8][3JAB][9913] = 4
count[12E8][3JBQ][9913] = 2    

```

* Output of the script should be as follows:

```
12E8:12E8,10090=4;3JAB,9913=4;3JBQ,9913=2
```

In [37]:

def BoaOutput2Text(Boa_file):
    
    output_file = Boa_file+"_converted"
    taxSet = set()
    current_id = -1
    txt_row =""
    with open(Boa_file, "r") as f, open(output_file, "w") as converted_file:
        for line in f:
            # print(line)
            
            words = line.split("[") # ['count', '12E8]', '3JAB]', '9913] = 4']
            row_id = words[1][:-1]
            seq_id = words[2][:-1]
            tax_id = words[3][:words[3].index("]")]
            tax_count = line[line.index("= ") + 2 :]

            if (current_id == -1):
                current_id = row_id
                txt_row = row_id +":"

            # while the new line is in the same cluster add the tax ids to the set
            if current_id == row_id:
                # count number of tax ids for each protein or cluster here
                if tax_id !='' and tax_id !='0':
                    taxSet.add(tax_id)
                    txt_row += seq_id + "," + tax_id +"=" + tax_count.rstrip() + ";"

            else:

                current_id = row_id
                txt_row = txt_row[:-1]

                # print(txt_row)
                if txt_row.count(';') > 0 :
                    converted_file.write(txt_row + "\n")

                txt_row = row_id + ":"
                taxSet.clear()
                if tax_id !='' and tax_id !='0':
                    taxSet.add(tax_id)
                    txt_row += seq_id + "," + tax_id+"="+tax_count.rstrip()+";"

    txt_row = txt_row[:-1]
    # print(txt_row)
    with open(output_file, "a") as converted_file:
        if txt_row.count(';') > 0:
            converted_file.write(txt_row+ "\n")


Boa_file ="part-r-00000"
BoaOutput2Text(Boa_file)


FileNotFoundError: [Errno 2] No such file or directory: 'part-r-00000'

FileNotFoundError: [Errno 2] No such file or directory: 'part-r-00000'

In [35]:
import pandas as pd
df = pd.read_fwf("BoaOutput_converted", header = None)
df

,0
0,"12E8:12E8,10090=4;3JAB,9913=4;3JBQ,9913=2"
1,"13PK:13PK,5691=4;16PK,5691=1"
2,"148L:148L,562=1;180L,10665=2;1QTV,10665=1"
3,"150L:150L,10665=4;256L,10665=1"
4,"15C8:15C8,10090=2;25C8,10090=1;35C8,10090=1"
